<a href="https://colab.research.google.com/github/netrunnr/PHYS454/blob/master/Coordinates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Coordinates & Sky Motion

In [0]:
#imports all the needed libraries, no output expected from this cell but you must run this for later code to work
from astropy import units as u
from astropy.coordinates import SkyCoord, EarthLocation
from astropy import coordinates as coord
from astropy.coordinates.tests.utils import randomly_sample_sphere
from astropy.time import Time
from astropy.coordinates import AltAz
from astropy.coordinates import get_sun

import datetime

from geopy.geocoders import Nominatim

import matplotlib.pyplot as plt

from datetime import datetime, timedelta
from pytz import timezone
import pytz

import matplotlib.dates as mdates

import numpy as np

Material gratefully adapted from multiple sources including: http://adrian.pw/blog/yearly-sun-graph/, https://docs.astropy.org/en/stable/coordinates/, and from the course text.

The **Celestial Sphere** is a model used to help us understand the positions of objects in the sky and how they move. Imagine a line projecting out along Earth's North pole and intersecting a sphere at a point. That point on the celestial sphere is the **North Celestial Pole**. The projection out of the Earth's equator makes a circle on the sphere, this is the **Celestial Equator**. The sphere is independent of any idea of distance from the Earth, which is helpful because we do not need to know about the distance to an astronomical object in order to talk about the positions and movements of objects in the sky. 

The stars are so distant that we can assume, for the purposes of this model, that they don't move relative to one another. They can be thought of as fixed into the sphere or painted on the sphere. Since the stars are in reality at many distances from Earth this is an inaccurate representation, but nevertheless the celestial sphere is a useful tool.

The Ecliptic represents the plane of the Solar System, or the plane defined by the Earth's orbit. It cuts the celestial sphere, deviating at maximum by the tilt of the Earth's axis, 23.5 degrees. The Sun and planets approximately travel along the ecliptic. 

Fig 1: The Celestial Sphere

![The Celestial Sphere with Earth centered within it and the North Celestial Pole at top](http://hyperphysics.phy-astr.gsu.edu/hbase/Solar/imgsol/eclipr.gif)

Like latitude and longitude are used as coordinates on the surface of near-spherical Earth, stars or other celestial objects have coordinates on the celestial sphere. Right Ascension ($\alpha$) is the equivalent of longitude, while Declination ($\delta$) is . This means that Right Ascension is measured *along* the celestial equator, while declination is measured *above* the celestial equator (Fig 2).

Fig 2: Celestial Sphere including lines of Right Ascension and Declination

![Celestial Sphere including lines of Right Ascension and Declination](https://www.researchgate.net/profile/Raghav_Maini/publication/305044094/figure/fig3/AS:669950733742082@1536740110163/The-celestial-sphere-with-grid-lines-of-right-ascension-and-declination.ppm)


In [0]:
#Example of grabbing the ra and dec coordinates for an object by name, in this case of the Andromeda Galaxy, or Messier 31 (M31)
m31 = SkyCoord.from_name('M31')
print(m31)

<SkyCoord (ICRS): (ra, dec) in deg
    (10.68470833, 41.26875)>


In [0]:
#Why can't we grab coordinates for the Sun?
sun = SkyCoord.from_name('Sun')
print(sun)

NameResolveError: ignored

Fig 3: Analemma

![alt text](https://scienceblogs.com/files/startswithabang/files/2009/08/analemma.gif)

![alt text](https://upload.wikimedia.org/wikipedia/commons/thumb/8/8c/Analemma_fishburn.tif/lossy-page1-560px-Analemma_fishburn.tif.jpg)

**QUESTION: What causes the Sun to vary in declination?**

The idea of the celestial sphere is often used in combination with a model of an observer's **horizon**. In Fig 1, the North Celestial Pole is at the top, but in our horizon model, we see that is not necessarily the case. The horizon is the circle which separates the part of the sky we can see from the part of the sky we can't see due to it being blocked by the Earth. For this example we take the whole celestial sphere and turn it in such a way that it represents the sky as it appears at a particular location.

Fig 4: The Horizon Model

![Diagram with observer at center of sphere representing the celestial sphere, with North Celestial Pole at about a 45 degree angle above the horizon](https://physics.weber.edu/schroeder/ua/CelestialSphere.png)

The celestial sphere is useful because it not only tells us about one sky we might see at a particular location, but also how that sky changes over the course of a night or over the course of a year. For example, Earth's rotation can be represented by rotating the celestial sphere around an axis going through the celestial poles. From a given location, the stars my rise and set. The sense of rotation can be seen with the arrows by the celestial poles in Fig 2, and show that for a mid Northern latitude like the one pictured, the stars which rise will do so somewhere within the Eastern part of the sky (e.g. Northeast or Southeast) and set somewhere within the Western part of the sky (e.g. in the Northwest or Southwest). The **Zenith** of an observer is the point directly overhead, and the **Celestial Meridian** is the half circle going from North on the horizon, through the zenith, to the South horizon. An object **transits** when it passes the meridian.


**QUESTION: If the declination of the Sun is 10° north and I observe it to transit 22° south of my zenith, where am I and when is it?**

In addition to defining a coordinate system to the stars, we can define a coordinate system relative to an observer. **Altitude** is the angle above the horizon perpendicular to the horizon, while **Azimuth** is measured in degrees East of North, parallel to the horizon (Fig 5).

Fig 5: Altitude and Azimuth

![A model showing how the location of a star is defined by the Altitude above the horizon and Azimuth, the angle East of North](https://astronavigationdemystifieddotcom.files.wordpress.com/2015/11/altitude-and-azimuth-mod.jpg)

**QUESTION: I am located at 47° north latitude.  I see a star transit 20° south of my zenith.  What is its declination? You may wish to draw a diagram of this scenario.**




In [0]:
#this shows an example of how to use SkyCoord to grab the altitude and azimuth of an object (in this case the Andromeda Galaxy, also known as M31) 
#as it appears for an observer at a specific location (in this case Bear Mountain in New York)

m31 = SkyCoord.from_name('M31')
bear_mountain = EarthLocation(lat=41.3*u.deg, lon=-74*u.deg, height=390*u.m)
utcoffset = -4*u.hour  # Eastern Daylight Time
time = Time('2019-8-27 22:00:00') - utcoffset
m31altaz = m31.transform_to(AltAz(obstime=time,location=bear_mountain))

print("M31's Altitude = {0.alt:.4}".format(m31altaz))
print("M31's Azimuth = {0.az:.4}".format(m31altaz))

M31's Altitude = 32.98 deg
M31's Azimuth = 61.4 deg


**QUESTION: Assuming the front of the room is North, point to where you think M31 would be in your sky.**



In [0]:
#This shows a trick to grab latitude and longitude from a named location using geolocator
geolocator = Nominatim(user_agent="phys_454")
New_York_City = geolocator.geocode("New York, NY")
print(New_York_City.latitude,New_York_City.longitude)

40.7127281 -74.0060152


In [0]:
#If you were not able to automatically look up the lat and long of your location, you can replace loc.latitude and loc.longitude with numerical values
New_York_City_earthlocation = EarthLocation(lat=New_York_City.latitude*u.deg, lon=New_York_City.longitude*u.deg, height=0*u.m)

utcoffset = -4*u.hour  # Eastern Daylight Time, may need to adjust for location
time = Time('2019-8-27 11:30:00') - utcoffset

#Defines a frame of the observer
altaz = AltAz(obstime=time, location=New_York_City_earthlocation)

#get_sun gets the general coordinates of the Sun at time = "time" and then transform_to(altaz) calculates altitude and azimuth for a given frame
sun_coords = get_sun(time).transform_to(altaz)

#print result, with format specified as showing two decimal places
print("Sun's Altitude = {0.alt:.4}".format(sun_coords))

Sun's Altitude = 53.71 deg




Fig 6: Solar and Siderial Day

![Diagram showing that the siderial day is slightly shorter than the solar day](http://www.celestialnorth.org/FAQtoids/SolSid_small.gif)

The Siderial day refers to the time it takes for the stars to reach the same position in the sky, while Solar day refers to the time it takes the Sun to reach the same position in the sky. Because the Earth must rotate slightly more due to the distance it has traveled in orbit, the solar day is longer than the siderial day by about 4 min. This relates to the Sun appearing to move relative to the background stars (Fig 6).

$\omega_{siderial}(t)$ is the angular velocity of rotation around it's own axis with reference to the "fixed stars"


$\omega_{Earth Sun}(t)$ is the angular velocity of it's orbit around the Sun, also with reference to the "fixed stars"


Then we have


$\omega_{siderial}(t) = \omega_{solar}(t) + \omega_{Earth Sun}(t)$


Using the definition $\omega = \frac{2\pi}{P}$ where P is the period, we get


$\frac{2\pi}{P_{siderial}} = \frac{2\pi}{P_{solar}} + \frac{2\pi}{P_{Earth Sun}}$


$P_{solar}$ is about 1 day, $P_{Earth Sun}$ is about 365 days, so $P_{Earth Sun} >> P_{solar}$


$P_{sid} = (\frac{1}{P_{solar}} + \frac{1}{P_{Earth Sun}})^{-1} = P_{solar} (1 + \frac{P_{Solar}}{P_{Earth Sun}})^{-1} \approx P_{Solar} (1 - \frac{P_{solar}}{P_{Earth Sun}})$


The difference between the solar and siderial day is then $P_{solar} - P_{siderial} \approx P_{solar} (\frac{P_{solar}}{P_{Earth Sun}}) \approx 1 day (\frac{1}{365})(\frac{24 hr}{1 day})(\frac{60 min}{1 hr}) \approx 4 min$

This all assumed that $\omega_{siderial}$ and $\omega_{Earth Sun}$ are constant.

**QUESTION: Why might $\vec{\omega}_{siderial}$ and $\vec{\omega}_{Earth Sun}$ vary slightly over time?**

We have the **apparent solar time**, measured by the Sun's actual position relative to the observer's meridian. This is the time measured by a sundial. There is also the **mean solar time**, which makes the assumption that the Sun travels steadily around the celestial equator at a constant rate, which is not entirely true. This is what your clocks measure.

**QUESTION: Referring to Figure 3, what the maximum difference between the mean solar time and the apparent solar time? When does this occur? What physically accounts for this?**